In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys
sys.path.append('../src')

import constants
from utils import train_test_split

# Read data

In [ ]:
ratings = pd.read_csv(constants.RATINGS_PATH_SANDBOX, parse_dates=['timestamp'])

# Data preprocessing

In [ ]:
# In train propouses we will use only 30% of all ratings dataset
rand_userIds = np.random.choice(ratings['userId'].unique(),
                                size=int(len(ratings['userId'].unique())*0.3),
                                replace=False)

ratings = ratings.loc[ratings['userId'].isin(rand_userIds)]
print('There are {} rows of data from {} users'.format(len(ratings), len(rand_userIds)))

### Train-test split

In [ ]:
train_ratings, test_ratings = train_test_split(ratings)

In [ ]:
train_ratings.sample(5)

In [ ]:
train_ratings[train_ratings['rating'] == 0.5]

# My custom dataset (MovieLense)

In [ ]:
class MovieLensTrainDataset(Dataset):
    """MovieLens PyTorch Dataset for Training

    Args:
        ratings (pd.DataFrame): Dataframe containing the movie ratings
    """

    def __init__(self, ratings: pd.DataFrame):
        self.ratings = ratings.copy()
        self.users, self.items, self.labels = self.get_dataset(self.ratings)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings):

        # Перевод оценок
        ratings['rating'] *= 2  # умножаем на 2, чтобы перейти к 10-балльной шкале

        users = ratings['userId'].values
        items = ratings['movieId'].values
        labels = ratings['rating'].values

        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)

# Model Architecture

In [ ]:
class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF)

        Args:
            num_users (int): Number of unique users (train + test)
            num_items (int): Number of unique items (train + test)
            train_ratings (pd.DataFrame): Dataframe containing the movie ratings for training
            val_ratings (pd.DataFrame): Dataframe containing the movie ratings for validating
    """

    def __init__(self, num_users, num_items, train_ratings, val_ratings):
        super().__init__()
        self.dataset = MovieLensTrainDataset(train_ratings)
        self.validation_dataset = MovieLensTrainDataset(val_ratings)

        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)

        input_size = 8 + 8  # user_embedded + item_embedded
        self.fc1 = nn.Linear(in_features=input_size, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=11)

        self.loss = nn.CrossEntropyLoss()

        self.dropout = nn.Dropout(0.5)


    def forward(self, user_input, item_input):
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layers
        vector = self.dropout(nn.ReLU()(self.fc1(vector)))
        vector = self.dropout(nn.ReLU()(self.fc2(vector)))

        pred = self.output(vector)

        return pred

    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        labels = labels.long()  # Ensure that labels are in the correct type
        predicted_logits = self(user_input, item_input)

        loss = self.loss(predicted_logits, labels)

        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        labels = labels.long()
        predicted_logits = self(user_input, item_input)

        loss = self.loss(predicted_logits, labels)

        self.log('val_loss', loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
        return [optimizer], [scheduler]

    def train_dataloader(self):
        return DataLoader(self.dataset, batch_size=512, num_workers=5, persistent_workers=True)

    def val_dataloader(self):
        return DataLoader(self.validation_dataset, batch_size=512, num_workers=5, persistent_workers=True)


In [ ]:
num_users = ratings['userId'].max()+1
num_items = ratings['movieId'].max()+1

model = NCF(num_users, num_items, train_ratings, test_ratings)

In [ ]:
checkpoint_callback = ModelCheckpoint(dirpath=r'weights/',
                                      filename='{epoch}-{train_loss:.2f}',
                                      monitor="train_loss")
trainer = pl.Trainer(max_epochs=5,
                    devices="auto", accelerator="auto",
                    logger=False,
                    callbacks=[checkpoint_callback],
                    fast_dev_run=False,)

In [ ]:
trainer.fit(model)

In [ ]:
trainer.save_checkpoint(r"weights/NCF_FINAL_epoch.ckpt")

In [ ]:
model = NCF.load_from_checkpoint(r"../src/weights/epoch=3-val_loss=1.79.ckpt",
                                 num_users=num_users,
                                 num_items=num_items,
                                 train_ratings=train_ratings,
                                 val_ratings=test_ratings)

In [ ]:
model = model.to('cpu')

In [ ]:
user_id, item_id, labels = model.validation_dataset[532]
user_id = user_id.unsqueeze(0)
item_id = item_id.unsqueeze(0)
predicted_logits = model(user_id, item_id)
predicted_probs = nn.Softmax(dim=1)(predicted_logits)
predicted_class = torch.argmax(predicted_probs, dim=1)
predicted_class

In [ ]:
labels